In [12]:
from datasets import load_dataset
dataset = load_dataset('docred', trust_remote_code=True)
ds = dataset['train_annotated']


Zest Airways , Inc. operated as AirAsia Zest ( formerly Asian Spirit and Zest Air ) , was a low - cost airline based at the Ninoy Aquino International Airport in Pasay City , Metro Manila in the Philippines .
It operated scheduled domestic and international tourist services , mainly feeder services linking Manila and Cebu with 24 domestic destinations in support of the trunk route operations of other airlines .
In 2013 , the airline became an affiliate of Philippines AirAsia operating their brand separately .
Its main base was Ninoy Aquino International Airport , Manila .
The airline was founded as Asian Spirit , the first airline in the Philippines to be run as a cooperative .
On August 16 , 2013 , the Civil Aviation Authority of the Philippines ( CAAP ) , the regulating body of the Government of the Republic of the Philippines for civil aviation , suspended Zest Air flights until further notice because of safety issues .
Less than a year after AirAsia and Zest Air 's strategic allian